# Goals: 
In this colab you will:
- Learn what DNA sequencing is, and why we care about it.
- Understand what the output from next generation sequencing looks like.
- Understand the difference between *de novo* assembly and reference based alignment.
- Perform your own *de novo* assembly of the SARS-CoV-2 genome!
- Perform your own reference-based alignment of the SARS-CoV-2 genome!
- View and understand your alignment using IGVviewer.

**Note: This colab does not use any machine learning, but rather teaches some fundamental concepts of genomics/bioinformatics. It is intended for those who are interested in using the state-of-the-art technologies that graduate students, professors, and companies use to analyze DNA.**

In [1]:
#@title # Install libraries - ignore warnings/errors.
%%bash 

# Make a local/temp directory called 'software' and download the needed software into it.
mkdir software
mkdir data
mkdir results
cd software

# Download/install required software packages (bwa, SPAdes, and samtools) -- 
# this just installs it to your temporary/local/cloud instance NOT to your 
# permanent gDrive.
git clone https://github.com/lh3/bwa.git
wget -O samtools.tar.bz2 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/samtools-1.3.1.tar.bz2'
tar -xjvf samtools.tar.bz2 
mv samtools-1.3.1 samtools
rm *.bz2
wget -O SPAdes-3.12.0-Linux.tar.gz 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/SPAdes-3.12.0-Linux.tar.gz'
tar -xzf SPAdes-3.12.0-Linux.tar.gz
mv SPAdes-3.12.0-Linux SPAdes
rm *.gz
chmod -R 777 '/content/software'
cd bwa; make
cd /content/software/samtools
configure; make; make install


samtools-1.3.1/
samtools-1.3.1/aclocal.m4
samtools-1.3.1/AUTHORS
samtools-1.3.1/bam.c
samtools-1.3.1/bam.h
samtools-1.3.1/bam2bcf.c
samtools-1.3.1/bam2bcf.h
samtools-1.3.1/bam2bcf_indel.c
samtools-1.3.1/bam2depth.c
samtools-1.3.1/bam_addrprg.c
samtools-1.3.1/bam_aux.c
samtools-1.3.1/bam_cat.c
samtools-1.3.1/bam_color.c
samtools-1.3.1/bam_endian.h
samtools-1.3.1/bam_flags.c
samtools-1.3.1/bam_import.c
samtools-1.3.1/bam_index.c
samtools-1.3.1/bam_lpileup.c
samtools-1.3.1/bam_lpileup.h
samtools-1.3.1/bam_mate.c
samtools-1.3.1/bam_md.c
samtools-1.3.1/bam_plbuf.c
samtools-1.3.1/bam_plbuf.h
samtools-1.3.1/bam_plcmd.c
samtools-1.3.1/bam_quickcheck.c
samtools-1.3.1/bam_reheader.c
samtools-1.3.1/bam_rmdup.c
samtools-1.3.1/bam_rmdupse.c
samtools-1.3.1/bam_sort.c
samtools-1.3.1/bam_split.c
samtools-1.3.1/bam_stat.c
samtools-1.3.1/bam_tview.c
samtools-1.3.1/bam_tview.h
samtools-1.3.1/bam_tview_curses.c
samtools-1.3.1/bam_tview_html.c
samtools-1.3.1/bamshuf.c
samtools-1.3.1/bamtk.c
samtools-1.3.1/

Cloning into 'bwa'...
--2022-06-29 16:45:39--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/samtools-1.3.1.tar.bz2
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 64.233.189.128, 74.125.23.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4030072 (3.8M) [application/octet-stream]
Saving to: ‘samtools.tar.bz2’

     0K .......... .......... .......... .......... ..........  1% 20.5M 0s
    50K .......... .......... .......... .......... ..........  2% 30.5M 0s
   100K .......... .......... .......... .......... ..........  3% 7.18M 0s
   150K .......... .......... .......... .......... ..........  5% 33.2M 0s
   200K .......... .......... .......... .......... ..........  6% 5.98M 0s
   250K .......... .......... .......... .......... ..........  7% 29.3M

In [2]:
#@title # Set up notebook
# import gdown
# paths = {'SRR11140744_ILLUMINA.fastq': 'https://drive.google.com/uc?id=1WQ-NS8yloaAORpUwTBYA_Sx2kQ7jhhSK',
#          'SRR10948474_OXFORD_NANOPORE.fastq': 'https://drive.google.com/uc?id=1y7B2d8ie8o_hB0kvJXEgevaDv4TOgVmX',
#          'NC_045512.2.fasta': 'https://drive.google.com/uc?id=1T9en64lqJfJLMX4dLykYeT1GKhEYoBD8',
#          'NC_045512.2.fasta.amb':'https://drive.google.com/uc?id=1fKRqfhMRK3H6Nx4gCpCf85ihf7EXfjF1',
#          'NC_045512.2.fasta.ann':'https://drive.google.com/uc?id=1nGhnNOmt46T_ucW0if68gtAqQSXrPsf5',
#          'NC_045512.2.fasta.bwt':'https://drive.google.com/uc?id=1ZRR1mwNDl11QHXAjkLLLXmowtW5teUeT',
#          'NC_045512.2.fasta.pac':'https://drive.google.com/uc?id=1cBvy8uRj7zrbcsGtQM4TWsrlbDgkYmCk',
#          'NC_045512.2.fasta.sa':'https://drive.google.com/uc?id=10QaiPPcwyY66DAGDgzjOBbc22GTdzczm',
# }

# for k in paths.keys():
#   gdown.download(paths[k], 'data/' + k, True)

!wget -P 'data' 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/SRR11140744_ILLUMINA.fastq'
!wget -P 'data' 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/SRR10948474_OXFORD_NANOPORE.fastq'
!wget -P 'data' 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/NC_045512.2.fasta'
!wget -P 'data' 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/NC_045512.2.fasta.amb'
!wget -P 'data' 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/NC_045512.2.fasta.ann'
!wget -P 'data' 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/NC_045512.2.fasta.bwt'
!wget -P 'data' 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/NC_045512.2.fasta.pac'
!wget -P 'data' 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/NC_045512.2.fasta.sa'


--2022-06-29 16:46:36--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/SRR11140744_ILLUMINA.fastq
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 64.233.189.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347466108 (331M) [application/octet-stream]
Saving to: ‘data/SRR11140744_ILLUMINA.fastq’

SRR11140744_ILLUMIN 100%[===================>] 331.37M  68.3MB/s    in 4.9s    

2022-06-29 16:46:41 (68.3 MB/s) - ‘data/SRR11140744_ILLUMINA.fastq’ saved [347466108/347466108]

--2022-06-29 16:46:41--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/SRR10948474_OXFORD_NANOPORE.fastq
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.

# Sequencing

***Sequencing*** is the process of reading the DNA or RNA of an organism.  There are several types of sequencing machines that take advantage of the unique chemical properties of A, T, C, and G to read through a piece of piece of DNA and learn it's bases.

There are two main types of sequencing: sequencing by synthesis and nanopore-based sequencing.



## Sequencing by Synthesis

In sequencing-by-synthesis, a special enzyme adds glowing complementary nucleotides to a piece of DNA that are then read by a camera.

[![](http://img.youtube.com/vi/3M0PyxFPwkQ/0.jpg)](https://www.youtube.com/watch?v=3M0PyxFPwkQ "Sequencing by Synthesis")

## Nanopore-based Sequencing


In nanopore-based sequencing, DNA is pulled through a special pore that generates a current when different bases go through the pore. The current is read by a voltmeter and converted into A,T,C, and G.

Click on the image below to watch an animation of nanopore-based sequencing.

[![](http://img.youtube.com/vi/RcP85JHLmnI/0.jpg)](https://www.youtube.com/watch?v=RcP85JHLmnI "Nanopore")

## Building Genomes


In the sequencing pipeline, DNA is first broken up into many small segments. Then the sequencer reads one segment at at time. (For reference, Sanger sequencing reads are often < 200 basepairs long, and the very first human genome sequenced (3 billion basepairs long) was done using Sanger sequencing!)

**How do we go from these small sequences to knowing the full genome of an organism or sample?**

There are two main approaches: *De novo* assembly and reference-based alignment. *De novo* assembly (you have no idea what the genome looks like) is like putting together a jigsaw puzzle with no photo of what it should look like in the end). Reference-based assembly (you know what the genome should look like and you want to find small variations in your sample) is like putting together a jigsaw puzzle while looking at the box of what the finished product should look like.



![De Novo vs Reference-Based Assembly](https://www.researchgate.net/profile/Christopher_Noune/publication/321179957/figure/fig4/AS:614377665343516@1523490458663/6-Comparison-of-reference-assembly-and-de-novo-assembly-A-Reference-assembly-maps.png)


## Reference-based Alignment


In reference-based alignment, we know what the genome of an organism theoretically looks like. Our goal is to map reads back to the part of the genome they belong to. We can then use the aligned reads to count how many reads aligned to which regions of the genome, to measure things like gene expression. Or we can identify regions of a genome in which our sample may have mutations.

## De novo assembly of an organism


In de novo assembly of an genome, we have no idea what the genome is! We must build it entirely from scratch. In the SARS-CoV-2 (the virus that causes COVID-19), the scientists in Wuhan who first discovered the virus performed [de novo assembly of the virus](https://pubmed.ncbi.nlm.nih.gov/32004165/). They took a throat swab from a patient. They sequenced *everything* in the swab. They threw out any reads that looked like human DNA (see reference-based assembly for how they did this). Then they pieced together the rest of the reads to produce the SARS-CoV-2 genome.

De novo assembly works best with the longest length of reads possible. Oxford nanopore is the hottest new technology for this purpose. In de novo assembly, we look for some amount of overlap between different reads and then overlay them. We keep doing this until we have 1 or more long lengths of unbroken sequences (also known as contigs). 


## **Exercise: Manually perform de novo asssembly and reference-based alignment from the reads in the activity slide deck.**

Make a copy of ```5_Bonus_Genome_Assembly_Activity```, open up the slide deck and move around the reads in order to perform de novo assembly and reference based alignment.

Record the mutations you found in your reference based alignment, and the sequence in your de novo assembly.

In [3]:
mutation_location  =  '14' #@param {type:"string"}
mutation_base  =  'G switched to T' #@param {type:"string"}
de_novo_sequence  =  'CGGGTAGCCACTGACGTAGCCGGTTGTCCACTT' #@param {type:"string"}

print('Mutation Location: 14th position')
print("Mutation Base: G switched to T")
print("De novo sequence: CGGGTAGCCACTGACGTAGCCGGTTGTCCACTT")

Mutation Location: 14th position
Mutation Base: G switched to T
De novo sequence: CGGGTAGCCACTGACGTAGCCGGTTGTCCACTT


---

# Reference-Based Alignment -- Do it yourself!

Situation: We know what the genome of an organism *should* look like. We sequence a sample and we want to know how it's genome compared to the theoretical genome. What are some examples of situtations where this might be the case?



The SARS-CoV-2 virus is actively mutating, so are curious which mutations a specific evolutionary lineage of SARS-CoV-2 has accumulated. We sequence the SARS-CoV-2 strain of interest (this one is from Australia) and compare it to our reference genome to find mutations.

### What we need
- The reads from the sequencer (fastq files).
- One or more [reference genomes](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2) (fasta files).

## Step 1: Examine your reference genome(s).

We are using [this reference genome](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2) (one of the first samples of SARS-CoV-2). What does a reference genome look like?

In [4]:
#@title #### **Exercise: Examine your reference genome.**
%%bash

# Show us what the reference genome file looks like (first 30 lines).
head -n 30 data/NC_045512.2.fasta

>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA
CGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAAC
TAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTG
TTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTC
CCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTAC
GTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGG
CTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGAT
GCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTC
GTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCT
TCTTCGTAAGAACGGTAATAAAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGACTTA
GGCGACGAGCTTGGCACTGATCCTTATGAAGATTTTCAAGAAAACTGGAACACTAAACATAGCAGTGGTG
TTACCCGTGAACTCATGCGTGAGCTTAACGGAGGGGCATACACTCGCTATGTCGATAACAACTTCTGTGG
CCCTGATGGCTACCCTCTTGAGTGCATTAAAGACCTTCTAGCACGTGCTGG

## Step 2: Examine your reads.

We'll be using the reads from a SARS-CoV-2 [sample](https://trace.ncbi.nlm.nih.gov/Traces/sra/?run=SRR11140744) taken from University of Wisconsin - Madison.

In [5]:
#@title #### **Exercise: Examine your reads.**
%%bash

SRR_ID=SRR11140744_ILLUMINA
head -n 30 data/${SRR_ID}.fastq

@SRR11140744.1/1
ACATAGGGCTGTTCAAGTTGAGGCAAAACGCCTTTTTCAACTTCTACTAAGCCACAAGTGCCATCTTTAAGATGTTGACGTGCCTCTGATAAGACCTCCTCCACGGAGTCTCCAAAGCCACGTACGAGCACGTCGCGAACCTGTAAAACAGGCAAACTGAGTTGGACGTGTGTTTTCTCGTTGAAACCAGGGACAAGGCTCTCCATCTTACCTTTCGGTCACACCCGGACGAAACCTAGATGTGCTGATGA
+
BCCCCFFFFFCFGGGGGGGGGGHGGHHHHGGGHGHHHHHHHHHHHHHHHHHHHHHHHGHHHHHHHHHHHHHHHHHHHHHGGGHHHHHGHHGHHHHHHHHHHHHHGGGGGHHHHHHHHHHHHGHHHGGGGGHGHHGGGGGGGHHHHHHHHHHHGGHHHHHFHHHHHHHGGGHHHHHHHHHGGGHHHHHHHHGGGGGGGFGGGGGGGGGGGGGGGGGGGGGGFFFFFFFFFDFFFFFFFFFFFFFFFFFFFFB
@SRR11140744.1/2
ACAGGACACGAGTAACTCGTCTATCTTCTGCTGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAA
+
ABAAAFBFFBDBGGGGGGGGGGHHHHHHHHHHCHGHGGGHHHGGHGGHGHGGGHFHHHHHHHHGGGGGHHHHHHHHHFHHHHGHHHGHGGGGGEFGDGHHGFGGGHHHHHGHHGGHHFHHHHGHHHHHHHHHHHHHHGFFGGHHHHHHGGHHGGHHHHHEGHHHHHHHGHHGHHFHHHHHGGGGGGGGGGGGAGGG9BEFFFFFFF

 ### Fields to care about.


 `@SRR11494554.6`: This is just the READ_ID, used so we can remember which sequence came from which read.

`ACCTTGAAGTGTTATCATTAG....`: This is the string of basepairs that the machine output it sequence that piece of DNA.

`/AAAAE6EEEEEEEEEEEEEEEEEEEEEEEEEEEEE`: This is the read quality. This is code that the machine outputs to describe how confident it was in each of the basepairs it called.




## Step 3: Align your reads to the reference genome.




In [ ]:
#@title #### **Exercise: Align your reads to the reference genome.**

%%bash
#################################################################
##### SETTING UP FILE DIRECTORIES/PERMISSIONS -- IGNORE ########
export PATH='/content/software/bwa':$PATH
DATA_DIR=data
RESULTS_DIR=results
#################################################################
#################################################################
SRR_ID=SRR11140744_ILLUMINA
REFERENCE_GENOME=NC_045512.2.fasta
cp ${DATA_DIR}/${REFERENCE_GENOME} ${RESULTS_DIR}/${REFERENCE_GENOME}


# Align reads to reference genome.
bwa index ${RESULTS_DIR}/${REFERENCE_GENOME} # We must index the reference file for bwa to work.
echo "DONE CREATING INDEX."

# Perform alignment algorithm (burrows-wheeler algorithm).
bwa mem ${RESULTS_DIR}/${REFERENCE_GENOME} -verbose -o ${RESULTS_DIR}/${SRR_ID}.bam ${DATA_DIR}/${SRR_ID}.fastq  

## Step 4: Examine your mapped reads.

In [6]:
#@title #### **Exercise: Examine your mapped reads.**
%%bash
#################################################################
##### SETTING UP FILE DIRECTORIES/PERMISSIONS -- IGNORE ########
SRR_ID=SRR11140744_ILLUMINA
#################################################################
#################################################################

# Split the aligned file into one file, 
# with reads that aligned to the reference (_mapped),
# and one file with reads that did not align (_unmapped).
samtools view  -f 0x04 -b -o results/${SRR_ID}_unmapped.bam results/${SRR_ID}.bam
samtools view  -F 0x04 -b -o results/${SRR_ID}_mapped.bam results/${SRR_ID}.bam

# Report the number of reads that were aligned and not aligned.
echo "# aligned to reference: " $(samtools view -c results/${SRR_ID}_mapped.bam) # The -c flag means "count" the number of reads.
echo "# unaligned to reference: " $(samtools view -c results/${SRR_ID}_unmapped.bam) # The -c flag means "count" the number of reads.

# Sort and index the files so that we can easily view them on IGV.
samtools sort -o results/${SRR_ID}_mapped_sorted.bam results/${SRR_ID}_mapped.bam 
samtools index -b results/${SRR_ID}_mapped_sorted.bam results/${SRR_ID}_mapped_sorted.bam.bai

# Show us what a .bam file looks like! (Uncompressed)
samtools view results/${SRR_ID}_mapped_sorted.bam | head -n 1000 | tail -n 10

# aligned to reference: 
# unaligned to reference: 


[E::hts_open_format] fail to open file 'results/SRR11140744_ILLUMINA.bam'
samtools view: failed to open "results/SRR11140744_ILLUMINA.bam" for reading: No such file or directory
[E::hts_open_format] fail to open file 'results/SRR11140744_ILLUMINA.bam'
samtools view: failed to open "results/SRR11140744_ILLUMINA.bam" for reading: No such file or directory
[E::hts_open_format] fail to open file 'results/SRR11140744_ILLUMINA_mapped.bam'
samtools view: failed to open "results/SRR11140744_ILLUMINA_mapped.bam" for reading: No such file or directory
[E::hts_open_format] fail to open file 'results/SRR11140744_ILLUMINA_unmapped.bam'
samtools view: failed to open "results/SRR11140744_ILLUMINA_unmapped.bam" for reading: No such file or directory
[E::hts_open_format] fail to open file 'results/SRR11140744_ILLUMINA_mapped.bam'
[bam_sort_core] fail to open 'results/SRR11140744_ILLUMINA_mapped.bam': No such file or directory
[E::hts_open_format] fail to open file 'results/SRR11140744_ILLUMINA_mapped_s

### Fields to care about.



 `@SRR11140744.3137`: This is the read id.

`NC_045512.2`: This is the reference genome (or chromsome) the read mapped to (sometimes we can use a set of multiple genomes as our reference, if we are not sure what organism a read may have come from - example: sequencing the bacteria in your gut.) In this case, we only used one reference genome so everything mapps to NC_045512 (the NCBI code for the SARS-CoV-2 reference sequence).

`170`: This is the position the beginning of the read aligned to on the reference genome.

`6S226M`: This is something called a CIGAR string that indicates how many mismatches there were between the read and the reference genome where it alined. means "6 basepairs were **S**oft-clipped (6 bases from one end of the sequence didn't align) and 226 basepairs **M**atched". Sometimes a read can align, but not perfectly match up to the reference genome, such as when the read has a mutation (or beacause of sequencing error).


## Step 5: Take a look on IGViewer.

In [7]:
#@title ####Run this cell to copy your data to google drive
from google.colab import drive
drive.mount('/content/drive')
!cp results/SRR11140744_ILLUMINA_mapped_sorted.* '/content/drive/My Drive'

Mounted at /content/drive
cp: cannot stat 'results/SRR11140744_ILLUMINA_mapped_sorted.*': No such file or directory


***Congrats!*** You made it throught the sometimes painful processes of genome alignment!  

(While you're here, go ahead and run steps 1 & 2 of De Novo Assmbly below -- step 2 can take a while, so we'll let it run while we examine our alignments)


![WHEE](https://bestanimations.com/Holidays/Fireworks/fireworks/ba-awesome-colorful-fireworks-animated-gif-image-s.gif)

You are now ready to see what your mapped reads look like using [IGViewer](https://igv.org/app/).  This is the most fun part of this lab :)
1. Go to IGViewer (integrated genome Viewer) webapp (link above).
2. Genomes-->SARS-CoV-2. (You could also input your fasta file from the Google Drive, but used the same reference genome for SARS-CoV-2 as IGV. IGV's is already annotated with the ORFs, etc so it is convenient to just use their preset!)
3. Track-->Google Drive. Search for ```SRR11140744_ILLUMINA_mapped_sorted``` and select both `SRR11140744_ILLUMINA_mapped.sorted.bam` and `SRR11140744_ILLUMINA_mapped.sorted.bam.bai` to select your final mapping and it's corresponding index file. It may take a couple minutes for IGV to load it.
4. Explore!! Understand what you are looking at. You should zoom in and scroll around!

### **Exercise: Answer the following questions about the alignment.**



In [ ]:
_1_ =  '' #@param {type:"string"}
_2_ =  '' #@param {type:"string"}


### **Exercise: Find and document the following features:**

In this particular SARS-CoV-2 sequence, find the location of:
1. A conserved single nucleotide polymorphism (SNP, ie a single basepair mutation)?
2. A loci where SARS-CoV-2 is highly variable (sometimes has a mutation and sometimes doesn't)?
3. A conserved deletion (hint: it is pretty small! You might have to zoom in and search around.)


In [ ]:
_1_ =  '' #@param {type:"string"}
_2_ =  '' #@param {type:"string"}
_3_ =  '' #@param {type:"string"}
print("1. Check out position 17,198")
print("2. Check out position 11,335")
print("3. Check out position 20,297")

# De Novo Assembly: Do it yourself!

Now, you will have the chance to perform de novo assembly and reference-based alignment and then view the SARS-CoV-2 genome, via the same state-of-the-art tools that researchers use to perform genome assembly and alignment.

What we neeed:

- Just the raw reads.

We will be de novo assembling a SARS-CoV-2 virus (the virus that causes COVID-19), just like the researchers in Wuhan did.

## Step 1: Examine your reads!

For *de novo* assembly, we will be using some [Oxford Nanopore reads](https://www.ncbi.nlm.nih.gov/sra/SRX7615553) of SARS-CoV-2, 
sequenced by HKU-Shenzhen Hospital.

In [ ]:
#@title #### **Exercise: Examine your reads.**
%%bash
SRR_ID=SRR10948474_OXFORD_NANOPORE
less data/${SRR_ID}.fastq | head -n 20 # This shows the first 20 lines of your fastq file. The fastq file is the output from the sequencer.

### Fields to care about:

 `@SRR10948474.30344`: This is just the READ_ID, used so we can remember which sequence came from which read.

`TGCGATAGTTCAGCACCTGTTTCCCACT....`: This is the string of basepairs that the machine output it sequence that piece of DNA. Oxford nanopore reads can be LONG!

`,("$+%2$4;E9=?@G?@@'BC.G<8FJA><'N<?@`: This is the read quality. This is code that the machine outputs to describe how confident it was in each of the basepairs it called.  Oxford nanopore reads are often error-prone.


## Step 2: De novo assemble your genome.

 This can take up to 10 minutes, so feel free to take a break/ask questions while this runs!

In [ ]:
#@title ### **Exercise: De novo assemble your genome**
%%bash
# IGNORE -- Setting up directory structure/permissions.
PATH='/content/software/SPAdes/bin':$PATH
DATA_FOLDER=data
RESULTS_DIR=results
SRR_ID=SRR10948474_OXFORD_NANOPORE

# Perform denovo assembly using SPAdes.
spades.py -s ${DATA_FOLDER}/${SRR_ID}.fastq -o ${RESULTS_DIR}/${SRR_ID}_spades

## Step 3: Examine your contigs.

In [ ]:
#@title ### **Exercise: Examine your contigs**
%%bash 
SRR_ID=SRR10948474_OXFORD_NANOPORE

cp results/${SRR_ID}_spades/contigs.fasta ${SRR_ID}_denovo.fasta
head -n 10 results/${SRR_ID}_denovo.fasta  # Show the first 10 lines of the de novo assembled genome.

### Fields to care about.


`NODE_1`: ID of the contig.

`length_23792`: The first contig is 23792 basepairs long.

`CTATTCTCAGCACCTGTTTCC...`: The assembled sequence.


We see that we assembled *nearly* (about 80%) of the entire SARS-CoV-2 genome!! (SARS-CoV-2 is ~30kbp, we got to ~24kbp). That's pretty good, given a single batch of sequencing reads.

# Wrapping up

***Nice job!*** We covered many new things in biology, genomics and machine learning! You've learned concepts that many students don't learn until their PhD!


![alt text](https://memeshappen.com/media/created/Job-WELL-done--meme-63067.jpg)

As a final exercise, brainstorm a few interesting projects you wish to do in the future! 


In [ ]:
#@title ###**Exercise: What are some other projects you wish to do in the future using biology, genomics, and machine learning?**
_1_ = "" #@param {type:"string"}
_2_ = "" #@param {type:"string"}
_3_ = "" #@param {type:"string"}

